In [1]:
import numpy as np 
from sklearn.linear_model import LinearRegression 
import torch 
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset , TensorDataset ,DataLoader 
from torch.utils.data.dataset import random_split 
from torch.utils.tensorboard import SummaryWriter


#### note that here we use index in get item because  , we dont know how big our dataset will ,  in the introduction 
### and we dont sent to gpu , sending directly huge data to gpu may creates us problme 

we dont want to load whole dataset in gpu because , doing  so ,  because it takes space in our precious graphis card

In [2]:
%run -i data_preparation/v0.py

In [3]:
# %load data_preparation/v0.py

import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_train = np.linspace(1, 10, 100)
x_train_tensor = torch.as_tensor(x_train, device=device, dtype=torch.float).view(-1 ,1 )
y_train = x_train * 2 + 1 
y_train_tensor = torch.as_tensor(y_train , device = device  , dtype = torch.float).view(-1 , 1 )


In [4]:
class customDataset(Dataset):
    def __init__(self , x_tensor , y_tensor):
        self.x  = x_tensor 
        self.y = y_tensor

    def __getitem__(self , index):
        return (self.x[index] , self.y[index])
    
    def __len__(self):
        return (len(self.x))
    
train_data = customDataset(x_train_tensor , y_train_tensor)
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = customDataset(x_train_tensor , y_train_tensor)
train_data[2]

(tensor(1.1818), tensor(3.3636))

In [5]:
# tensor dataset class
train_data = TensorDataset(x_train_tensor , y_train_tensor)
train_data[0]

(tensor(1.), tensor(3.))

In [6]:
train_loader = DataLoader( dataset= train_data , shuffle=True , batch_size = 16 )
train_loader


In [7]:
len(next(iter(train_loader)))
#their are 16 batches and this will print  1 batch


2

In [8]:
len(list(train_loader))

7

In [9]:
%%writefile data_preparation/v1.py
import torch 
import numpy as np
from torch.utils.data import TensorDataset , DataLoader

x_train = np.linspace(1 , 10 , 100)
x_train_tensor = torch.as_tensor(x_train).float().view(-1,1)
y_train = x_train *2 + 1 
y_train_tensor = torch.as_tensor(y_train).float().view(-1 , 1)

#build dataset
train_data = TensorDataset(x_train_tensor , y_train_tensor)


train_loader = DataLoader(
    dataset = train_data , 
    batch_size = 16 , 
    shuffle = True
)


Overwriting data_preparation/v1.py


In [10]:
%run -i data_preparation/v1.py

add mini batch gradent logic in model configuration

In [11]:
%run -i  data_preparation/v1.py
%run -i  model_configuration/v1.py

In [12]:
#model _tranning
n_epochs = 1000
losses = []
for epoch in range(n_epochs):
    mini_batch_losses = []
    for x_batch , y_batch in train_loader:
        # for number of instances
        mini_batch_losses = []
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = train_step_fn(x_batch , y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    losses.append(loss)

    if epoch % 10 == 0 :
        print(f'Epoch {epoch}  Loss : {loss}')

    

Epoch 0  Loss : 0.37308263778686523
Epoch 10  Loss : 0.48805946111679077
Epoch 20  Loss : 0.2552509903907776
Epoch 30  Loss : 0.12849591672420502
Epoch 40  Loss : 0.09309923648834229
Epoch 50  Loss : 0.05134318768978119
Epoch 60  Loss : 0.01755167916417122
Epoch 70  Loss : 0.0066045131534338
Epoch 80  Loss : 0.006432800553739071
Epoch 90  Loss : 0.004398713819682598
Epoch 100  Loss : 0.0009666635887697339
Epoch 110  Loss : 0.0017308444948866963
Epoch 120  Loss : 0.0015057220589369535
Epoch 130  Loss : 0.00033792603062465787
Epoch 140  Loss : 7.943905075080693e-05
Epoch 150  Loss : 0.0003559039905667305
Epoch 160  Loss : 6.264795229071751e-05
Epoch 170  Loss : 4.9136044253828004e-05
Epoch 180  Loss : 4.168749728705734e-05
Epoch 190  Loss : 5.8373356296215206e-05
Epoch 200  Loss : 2.3695567506365478e-05
Epoch 210  Loss : 1.2939080079377163e-05
Epoch 220  Loss : 8.297183740069158e-06
Epoch 230  Loss : 3.3871526738948887e-06
Epoch 240  Loss : 5.93708136875648e-07
Epoch 250  Loss : 1.532804

### we execute inner loop 100/16 = 6.25 = 7 times 
### thus , we are calling the train_step_fn 7 * 1000 i.e 7000 times 



In [13]:
### mini batch helper function


In [16]:
%%writefile model_training/v3.py
n_epochs = 200 
losses = [] 


def mini_batch(device , train_loader , train_step_fn):
    mini_batch_losses = [] 
    for x_batch , y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = train_step_fn(x_batch , y_batch)
        mini_batch_losses.append(mini_batch_loss)
    loss =np.mean(mini_batch_loss)
    return loss 



for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader , train_step_fn)
    losses.append(loss)


Writing model_training/v3.py


### remember  in batch gradient descent  , 
####  700 epcohs means  , 700 updates 

#### but here  , since our 100 len data is divinded into  16 batch i.e total 7 batches
#### tus on each epochs we are updating 7 times 
#### thus here we only need 10 epcohs to have 700 updates i.e(7 * 700) 

In [17]:
%run -i model_training/v3.py

In [18]:
#model configuration
model.state_dict()

OrderedDict([('0.weight', tensor([[2.0000]], device='cuda:0')),
             ('0.bias', tensor([1.0000], device='cuda:0'))])

## random split


In [30]:
%%writefile data_preparation/v2.py
import torch 
import numpy as np 
from torch.utils.data import TensorDataset , DataLoader , random_split 

torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = np.linspace(1 , 10 , 100)
y  = 2*x  + 10 
x_tensor = torch.as_tensor(x).float()
y_tensor = torch.as_tensor(y).float()

#build 
dataset = TensorDataset(x_tensor , y_tensor)

#perform split
ratio = 0.8 
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train 
train_data , val_data = random_split(dataset , [n_train,n_val])


#build a loader each set
train_loader = DataLoader(
    dataset = train_data ,  
    batch_size = 16 , 
    shuffle = True
)

val_loader = DataLoader(
    dataset = val_data , 
    batch_size = 16 )


Overwriting data_preparation/v2.py


In [31]:
%run -i data_preparation/v2.py